In [1]:
#libraries

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense

import keras.backend as K
from keras.callbacks import EarlyStopping
#from Keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

In [2]:
#Get the Dataset
df=pd.read_csv(r'C:\Users\varun\Downloads\Qmca9Y4Bp6s2FkVfRTNS6zZU4HvBd24sXgR2RsR1uBP5Re\dataset\train_dataset.csv')
df.head()

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.000000,0.000000,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
1,2018-04-29,DLT,0.310345,0.310345,0.137931,0.275862,0.206897,0.517241,0.344828,0.689655,...,0.206897,0.206897,0.379310,0.172414,0.241379,0.379310,0.241379,0.793103,0.517241,-0.005240
2,2018-04-29,ENJ,0.241379,0.068966,0.310345,0.344828,0.103448,0.103448,0.379310,0.206897,...,0.379310,0.344828,0.310345,0.482759,0.655172,0.448276,0.724138,0.724138,0.413793,-0.036471
3,2018-04-29,OMG,0.758621,0.965517,0.896552,0.965517,0.965517,0.965517,0.068966,0.275862,...,0.896552,0.931034,0.896552,0.931034,0.896552,0.896552,0.827586,0.827586,0.275862,-0.066945
4,2018-04-29,UFR,0.620690,0.413793,0.275862,0.413793,0.517241,0.362069,0.758621,0.586207,...,0.482759,0.275862,0.172414,0.551724,0.103448,0.103448,0.137931,0.965517,0.724138,0.020286


In [3]:
symbols=df["symbol"].unique()
symbols


array(['BTC', 'DLT', 'ENJ', 'OMG', 'UFR', 'VGX', 'DRGN', 'BAT', 'AGIX',
       'KEY', 'KNC', 'TRAC', 'PRO', 'PLAY', 'PAY', 'DCN', 'DAY', 'CAN',
       'XRP', 'SRN', 'SALT', 'REQ', 'QSP', 'TRST', 'USDT', 'MGO', 'GET',
       'FUN', 'PRE', 'ETH', 'SPANK', 'ZRX', 'POLL', 'FUEL', 'ANT', 'AST',
       'SNGLS', 'VEE', 'VIB', 'PLR', 'BRD', 'WABI', 'MTL', 'NGC', 'IDH',
       'VIBE', 'PPT', 'DADI', 'DNT', 'GNO', 'POWR', 'DENT', 'GRID', 'OCN',
       'WPR', 'EVE', 'DNA', 'DGD', 'WTC', 'GVT', 'CDT', 'GLM', 'POLY',
       'RCN', 'EXRN', 'MTN', 'TKN', 'AXP', 'MANA', 'CRPT', 'HGT', 'REM',
       'ABT', 'RDN', 'LOC', 'LUN', 'MCO', 'PRG', 'ENG', 'CND', 'TNB',
       'PLBT', 'TEL', 'ELF', 'ATL', 'MKR', 'PPP', 'UTK', 'POE', 'NCASH',
       'BKX', 'ELEC', 'STORM', 'SNT', 'CHP', 'QASH', 'CAS', 'SHIP', 'SAN',
       'BNT', 'DMT', 'STAR', 'BANCA', 'ZAP', 'IND', 'BERRY', 'SAI', 'B2B',
       'BLZ', 'RFR', 'NCT', 'HT', 'TEN', 'ODE', 'CBT', 'BAX', 'DATA',
       'SENC', 'MWAT', 'DXT', 'SMT', 'LINK', 'CRC', 'M

In [4]:
count=0
for (uniquerow), group in df.groupby(df.symbol):
    count+=1
    globals()[uniquerow] = group

In [5]:
BTC

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
0,2018-04-29,BTC,0.0,0.0,0.965517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.689655,0.655172,0.655172,0.724138,0.827586,0.827586,0.931034,0.448276,0.758621,0.024718
84,2018-05-06,BTC,1.0,0.0,0.981481,1.000000,0.981481,0.981481,0.000000,0.000000,...,0.388889,0.555556,0.537037,0.481481,0.777778,0.759259,0.722222,0.722222,0.685185,-0.101384
140,2018-05-13,BTC,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.426471,0.500000,0.397059,0.500000,0.750000,0.735294,0.705882,0.705882,0.838235,-0.024447
185,2018-05-20,BTC,0.0,0.0,0.988636,0.000000,0.011364,0.988636,0.988636,0.000000,...,0.250000,0.363636,0.397727,0.318182,0.522727,0.579545,0.500000,0.806818,0.863636,-0.144448
272,2018-05-27,BTC,0.0,0.0,0.000000,0.989362,0.000000,0.000000,0.989362,0.978723,...,0.095745,0.212766,0.287234,0.191489,0.595745,0.606383,0.553191,0.872340,0.361702,0.046671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83961,2022-07-17,BTC,1.0,1.0,1.000000,0.997122,1.000000,0.000000,1.000000,1.000000,...,0.548201,0.588489,0.615827,0.572662,0.755396,0.775540,0.717986,0.404317,0.635971,0.083959
84474,2022-07-24,BTC,0.0,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.381089,0.538682,0.561605,0.517192,0.603152,0.630372,0.578797,0.636103,0.474212,0.031658
85376,2022-07-31,BTC,0.0,0.0,0.998586,0.998586,0.000000,0.000000,0.998586,0.000000,...,0.265912,0.404526,0.458274,0.352192,0.519095,0.570014,0.438472,0.472419,0.335219,-0.007029
86082,2022-08-07,BTC,1.0,0.0,0.000000,1.000000,1.000000,0.998584,0.000000,1.000000,...,0.205382,0.296034,0.327195,0.254958,0.454674,0.498584,0.389518,0.337110,0.629603,0.048287


In [19]:
ATN

,date,symbol,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2003,feature_2004,feature_2005,feature_2006,feature_2007,feature_2008,feature_2009,feature_DC1,target,target_DC1
4976,2018-12-16,ATN,0.631696,0.502232,0.616071,0.680804,0.676339,0.651786,0.718750,0.468750,...,0.571429,0.401786,0.392857,0.424107,0.500000,0.491071,0.513393,0.799107,0.763393,0.334990
5159,2018-12-23,ATN,0.393939,0.534632,0.376623,0.517316,0.597403,0.547619,0.556277,0.616883,...,0.238095,0.337662,0.359307,0.333333,0.367965,0.367965,0.350649,0.761905,0.627706,-0.028946
5536,2018-12-30,ATN,0.740506,0.654008,0.759494,0.662447,0.784810,0.784810,0.740506,0.736287,...,0.337553,0.434599,0.468354,0.379747,0.459916,0.497890,0.426160,0.624473,0.109705,-0.095611
5649,2019-01-06,ATN,0.265690,0.495816,0.380753,0.525105,0.395397,0.514644,0.564854,0.606695,...,0.564854,0.514644,0.539749,0.485356,0.573222,0.606695,0.539749,0.112971,0.769874,-0.148778
5898,2019-01-13,ATN,0.754167,0.514583,0.733333,0.639583,0.762500,0.608333,0.708333,0.787500,...,0.362500,0.370833,0.354167,0.362500,0.454167,0.437500,0.454167,0.770833,0.383333,0.023228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51123,2021-06-20,ATN,0.731141,0.701161,0.698259,0.774662,0.754352,0.863636,0.882012,0.841393,...,0.967118,0.920696,0.808511,0.965184,0.655706,0.411992,0.936170,0.945841,0.628627,-0.142997
52078,2021-06-27,ATN,0.275047,0.507561,0.480151,0.412098,0.517958,0.584121,0.739130,0.748582,...,0.956522,0.969754,0.971645,0.962193,0.911153,0.905482,0.918715,0.635161,0.164461,-0.028105
52136,2021-07-04,ATN,0.505703,0.324144,0.517110,0.474335,0.395437,0.489544,0.581749,0.713878,...,0.954373,0.956274,0.952471,0.950570,0.802281,0.752852,0.842205,0.165399,0.005703,-1.170049
53138,2021-07-11,ATN,0.625472,0.626415,0.416981,0.628302,0.611321,0.502830,0.575472,0.680189,...,0.973585,0.954717,0.937736,0.966038,0.318868,0.190566,0.616981,0.005660,0.215094,-0.213517


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
cor=[[] for i in range(980)]

In [ ]:
for j in range (0,980):
    for i in range (0,2009):
        cor[j].append(globals()[symbols[j]]['feature_'+ str(i+1)].corr(globals()[symbols[j]]['target']))

In [8]:
index=[[] for i in range(980)]

In [9]:
for i in range (980):
    index[i] = np.abs(np.array(cor[i]))
    index[i] = np.argpartition(cor[i], -15)[-15:]

In [20]:
cor[0]

[0.057589542003998746,
 -0.005911653864813141,
 0.008987180323918815,
 0.0032052201048564342,
 0.09642164276120502,
 -0.015515723908313179,
 -0.004038524837560715,
 0.07119726591394648,
 0.009125472482731291,
 -0.0013264333317391439,
 0.016225912903367626,
 -0.01325530207179501,
 0.07768326703277086,
 -0.04785798506868309,
 -0.011597151700131895,
 0.0367680882887067,
 0.08960054638424056,
 0.039404302576653935,
 0.04165484890828178,
 0.011972306388100174,
 0.0006257622347860994,
 0.06470159729763873,
 -0.015473019413535738,
 -0.054380383186762075,
 -0.028128421325542188,
 0.04436758053920567,
 0.016603841525204697,
 0.03208361691693381,
 0.07963125784059658,
 0.0669574181683149,
 -0.04264698411636607,
 -0.08244789308314424,
 -0.02518841566926761,
 -0.11750988314140422,
 0.022213609200879697,
 0.0577834212024296,
 0.008865125636847982,
 0.035239788239248365,
 -0.002312534285065998,
 0.07457548823456923,
 -0.03842364498835134,
 -0.11387200575029745,
 -0.1082834037812084,
 -0.033886546398

In [11]:
index[1]

array([1594, 1005,  445, 1582, 1550, 1007, 1008, 1720,  768, 1562, 1614,
        767, 1610, 1599, 1004], dtype=int64)

In [44]:
features=[[] for i in range(980)]
for i in range(980):
    for j in index[i]:
        features[i].append('feature'+str(j+1))
        
        
        

In [ ]:
# keep in mind to run only once the above code

In [45]:
count

15

In [46]:
len(features[0])

15

In [49]:
features[18]

['feature618',
 'feature1539',
 'feature892',
 'feature1629',
 'feature1281',
 'feature1011',
 'feature442',
 'feature1538',
 'feature233',
 'feature608',
 'feature237',
 'feature1756',
 'feature239',
 'feature1758',
 'feature1765']

In [ ]:
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(df[features])
feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=df.index)
feature_transform.head()